In [2]:
#imports 
import polars as pl 
import gzip 
from lxml import etree
import pandas as pd 
import xmltodict
import glob
import os 
import time 
import polars as pl 
import duckdb
import shutil 
import pyarrow.parquet as pq
import pyarrow as pa

In [76]:
''' 
Flemme de le faire au propre : 
Ce fichier contient des avancées sur la task 1, un nouveau schema et des tests sur la task 2 

La V2 de task 1 traite en plus :
  - F Le cas où il n'y a qu'une seule ligne budget (pour caracsup et mtsup).
  - F Ajoute des chemins que l'on peut choisir
  - F Ajoute une verification depuis un plus grand parquet, ça pourra se faire via une db plus tôt, quand on en aura une
  - F Sert un café robusta pas fou mais buvable
  - F Ajoute de nouvelles métadonnées, dont l'année d'exercice, la nature de vote, le siret d'etab parfois different du premier siret etc. 
  - En cours, Fait le schema à cette étape pour éviter tout soucis après 
 '''

" \nFlemme de le faire au propre : \nCe fichier contient des avancées sur la task 1, un nouveau schema et des tests sur la task 2 \n\nLa V2 de task 1 traite en plus :\n  - F Le cas où il n'y a qu'une seule ligne budget (pour caracsup et mtsup).\n  - F Ajoute des chemins que l'on peut choisir\n  - F Ajoute une verification depuis un plus grand parquet, ça pourra se faire via une db plus tôt, quand on en aura une\n  - F Sert un café robusta pas fou mais buvable\n  - F Ajoute de nouvelles métadonnées, dont l'année d'exercice, la nature de vote, le siret d'etab parfois different du premier siret etc. \n  - En cours, Fait le schema à cette étape pour éviter tout soucis après \n "

In [3]:
def _recherche_id_dans_parquet(chemin_parquet_db):
 
 conduck = duckdb.connect(database=':memory:', read_only=False)
 source_duck = conduck.read_parquet(f'{chemin_parquet_db}')
 requete = conduck.execute(''' 
    SELECT DISTINCT Id_Fichier 
    FROM source_duck''')
 ligne_sql_int = [int(x[0]) for x in requete.fetchall()]
 conduck.close()
 return ligne_sql_int


def parse_fichier(chemin) : 
 '''Ouvre et parse le fichier gzip'''
 with gzip.open(chemin, 'rb') as fichier_ouvert : 
  fichier_xml_gzip = fichier_ouvert.read()
  fichier_xml = fichier_xml_gzip.decode('latin-1')
  fichier_dict = xmltodict.parse(fichier_xml)
 return fichier_dict

def recherche_id_fichier(chemin_parquet) :
 ''' Pas encore utilisée, permet de récupérer les ID dans le parquet contenant les 
 données déjà traitées '''
 conduck = duckdb.connect(database=':memory:', read_only=False)
 docubudg_t = conduck.read_parquet(chemin_parquet)
 requete_duckdb = ''' 
 SELECT
    DISTINCT Id_Fichier
 FROM 
    docubudg_t
 '''
 result_requete= conduck.execute(requete_duckdb).fetchdf()
 liste_id = result_requete['Id_Fichier'].to_list()
 conduck.close()
 return liste_id

def _isolement_id(fichier) : 
 '''Extrait l'id du nom du fichier pour la liste comprehension de securité

 ATTENTION, le premier split / va changer si on l'applique sur du minio '''
 val_id_fichier_source = fichier.split("\\")[-1].split('.')[0]
 if '-' in val_id_fichier_source : 
  val_id_fichier = val_id_fichier_source.split('-')[1]
 else : 
  val_id_fichier= val_id_fichier_source
 return val_id_fichier

def nettoyage_V(dataframe : pd.DataFrame) -> pd.DataFrame :
 ''' Permet de supprimer les @V des colonnes à l'exception de MtSup et CaracSup'''

 nettoyage = lambda x : str(x).replace("{'@V': '", "").replace("'}", "")
 for col in dataframe.columns : 
  if col in ['CaracSup', 'CaracSup'] : 
   dataframe[col] = dataframe[col].astype(str) 
  else :
   dataframe[col] = dataframe[col].apply(nettoyage)
 dataframe_propre = dataframe.reset_index(drop=True)
 return dataframe_propre

def transformation_mtsup(lignes_budget) :
 ''' Traite les MtSup, qu'ils soient sous forme de list ou de dict '''

 for i in lignes_budget:
  type_mtsup = i.get('MtSup')  # Permet de connaitre le type de MtSup
  if type_mtsup is not None:  # Vérifie si la clé 'MtSup' existe
    if isinstance(type_mtsup, dict):
      dict_mtsup = type_mtsup
      i['MtSup_1_Lib'] = {'@V': dict_mtsup.get('@Code', '')}
      i['MtSup_1_Val'] = {'@V': dict_mtsup.get('@V', '')}
    elif isinstance(type_mtsup, list):
      dict_mtsup = i['MtSup']
      mtsup_propre = {}
      for z, entry in enumerate(dict_mtsup, start=1):
        code = f'MtSup_{z}_Lib'
        valeur = f'MtSup_{z}_Val'
        mtsup_propre[code] = entry.get('@Code', '')
        mtsup_propre[valeur] = entry.get('@V', '')
        i.update(mtsup_propre)
 return lignes_budget

def transformation_caracsup(lignes_budget) :
 ''' Pareil que transfo_MtSup mais pour CaracSup'''

 for i in lignes_budget:
  type_caracsup = i.get('CaracSup')  # Permet de connaitre le type de CaracSup
  if type_caracsup is not None:  # Vérifie si la clé 'CaracSup' existe
    if isinstance(type_caracsup, dict):
      dict_caracsup = type_caracsup
      i['CaracSup_1_Lib'] = {'@V': dict_caracsup.get('@Code', '')}
      i['CaracSup_1_Val'] = {'@V': dict_caracsup.get('@V', '')}
    elif isinstance(type_caracsup, list):
      dict_caracsup = i['CaracSup']
      caracsup_propre = {}
      for z, entry in enumerate(dict_caracsup, start=1):
        code = f'CaracSup_{z}_Lib'
        valeur = f'CaracSup_{z}_Val'
        caracsup_propre[code] = entry.get('@Code', '')
        caracsup_propre[valeur] = entry.get('@V', '')
        i.update(caracsup_propre)      
 return lignes_budget

def task_1(chemin) : 
 ''' Traite les XML et les envoies sous format parquet,
   le nettoyage des @V est fait à cette étape'''
 
 for fichier in glob.glob(os.path.join(chemin, "*.gz")) : 
  try : 
   id_fichier = _isolement_id(fichier)
   dico = parse_fichier(fichier)
   metadonnees = dico['DocumentBudgetaire']['EnTeteDocBudgetaire']
   metadonnees['Id_Fichier'] = {'@V' : id_fichier}
   docbase = dico['DocumentBudgetaire']['Budget']['LigneBudget']
   if isinstance(docbase, list):
    for i in docbase : 
      i.update(metadonnees)
   elif isinstance(docbase, dict) : 
    docbase.update(metadonnees)
   docbase = transformation_caracsup(docbase)
   docbase = transformation_mtsup(docbase)
   df_base = pd.DataFrame(docbase)
   df_propre = nettoyage_V(df_base)
   df_propre.to_parquet(f'./fichiers/todo_parquet/{id_fichier}_pyarrow', engine='pyarrow')
   #shutil.move(fichier, './fichiers/done_xml/')
  except Exception as e : 
    print(f'Erreur fichier {fichier}, extraction impossible')
    #shutil.move(fichier, './fichiers/todo_xml/error/')
    print(e)
    continue  

In [10]:
requete = _recherche_id_dans_parquet()

In [11]:
requete

[613605,
 614411,
 614740,
 615694,
 616768,
 617175,
 618729,
 621571,
 623745,
 624343,
 624555,
 624841,
 627383,
 630892,
 631522,
 633463,
 635720,
 636383,
 636822,
 636994,
 638694,
 639770,
 640489,
 641507,
 642617,
 643209,
 643295,
 643974,
 645336,
 645664,
 646790,
 649425,
 650697,
 651048,
 651646,
 652711,
 655361,
 656757,
 656956,
 658400,
 660293,
 662814,
 666115,
 667119,
 668460,
 671544,
 673713,
 674277,
 675122,
 675613,
 676825,
 678719,
 682541,
 684931,
 685166,
 685641,
 686863,
 689888,
 692190,
 692978,
 694727,
 695382,
 696883,
 697108,
 697439,
 701368,
 702637,
 703095,
 705356,
 706803,
 707554,
 707746,
 709050,
 710337,
 710735,
 712307,
 712435,
 714673,
 716057,
 718987,
 719567,
 720093,
 722578,
 725092,
 727157,
 727813,
 729770,
 731869,
 733537,
 734042,
 735624,
 735993,
 736192,
 737177,
 740031,
 742638,
 743489,
 743628,
 744499,
 745762,
 745931,
 746131,
 748061,
 748556,
 748869,
 749438,
 750556,
 751029,
 751886,
 755399,
 756152,
 

TEST avec les fichiers n'ayant qu'une ligne budget (dict et pas list)

Erreur fichier ./fichiers/todo_xml\619027.xml.gz, extraction impossible
'str' object has no attribute 'update'
Erreur fichier ./fichiers/todo_xml\628857.xml.gz, extraction impossible
'str' object has no attribute 'update'
Erreur fichier ./fichiers/todo_xml\682109.xml.gz, extraction impossible
'str' object has no attribute 'update'
Erreur fichier ./fichiers/todo_xml\722795.xml.gz, extraction impossible
'str' object has no attribute 'update'
Erreur fichier ./fichiers/todo_xml\730073.xml.gz, extraction impossible
'str' object has no attribute 'update'
Erreur fichier ./fichiers/todo_xml\748147.xml.gz, extraction impossible
'str' object has no attribute 'update'
Erreur fichier ./fichiers/todo_xml\756300.xml.gz, extraction impossible
'str' object has no attribute 'update'
Erreur fichier ./fichiers/todo_xml\775775.xml.gz, extraction impossible
no element found: line 1, column 0
Erreur fichier ./fichiers/todo_xml\807134.xml.gz, extraction impossible
'str' object has no attribute 'update'
Erreur fichier ./fichiers/todo_xml\809162.xml.gz, extraction impossible
'str' object has no attribute 'update'
Erreur fichier ./fichiers/todo_xml\809290.xml.gz, extraction impossible
'str' object has no attribute 'update'
Erreur fichier ./fichiers/todo_xml\819789.xml.gz, extraction impossible
'str' object has no attribute 'update'
Erreur fichier ./fichiers/todo_xml\821461.xml.gz, extraction impossible
'str' object has no attribute 'update'
Erreur fichier ./fichiers/todo_xml\838663.xml.gz, extraction impossible
'str' object has no attribute 'update'

In [6]:
chemin_619027 = '..//../etl_hub_coll/fichiers/todo_xml/619027.xml.gz'
id_619027 = '619027'

In [8]:
dico_619027 = parse_fichier(chemin_619027)
metadonnees_619027 = dico_619027['DocumentBudgetaire']['EnTeteDocBudgetaire']
metadonnees_619027['Id_Fichier'] = {'@V' : id_619027}
metadonnees_619027 
docbase_619027 = dico_619027['DocumentBudgetaire']['Budget']['LigneBudget']
type(docbase_619027)

dict

In [9]:
docbase_619027

{'Nature': {'@V': '748'},
 'ArtSpe': {'@V': 'false'},
 'CodRD': {'@V': 'R'},
 'MtReal': {'@V': '248125.45'},
 'MtRAR3112': {'@V': '0'},
 'OpBudg': {'@V': '0'}}

In [12]:
docbase_619027 = transformation_mtsup(docbase_619027)

UnboundLocalError: cannot access local variable 'i' where it is not associated with a value

Modif = isolement id, recherche id parquet, transfo mtsup carac sup et modif script 

In [25]:
def _isolement_id_liste_comp(fichier) : 
 "Extrait l'id du nom du fichier pour la liste comprehension de securité"
 val_id_fichier = fichier.split("-")[-1].split('.')[0]
 return val_id_fichier

In [4]:
def transfo_mtsup_if_dict(ligne_budget_dict : dict) -> dict:
 ''' Traite les MtSup, qu'ils soient sous forme de list ou de dict '''
 type_mtsup = ligne_budget_dict.get('MtSup')  # Permet de connaitre le type de MtSup
 if type_mtsup is not None:  # Vérifie si la clé 'MtSup' existe
    if isinstance(type_mtsup, dict):
      dict_mtsup = type_mtsup
      ligne_budget_dict['MtSup_1_Lib'] = {'@V': dict_mtsup.get('@Code', '')}
      ligne_budget_dict['MtSup_1_Val'] = {'@V': dict_mtsup.get('@V', '')}
    elif isinstance(type_mtsup, list):
      dict_mtsup = ligne_budget_dict['MtSup']
      mtsup_propre = {}
      for z, entry in enumerate(dict_mtsup, start=1):
        code = f'MtSup_{z}_Lib'
        valeur = f'MtSup_{z}_Val'
        mtsup_propre[code] = entry.get('@Code', '')
        mtsup_propre[valeur] = entry.get('@V', '')
        ligne_budget_dict.update(mtsup_propre)
 return ligne_budget_dict

def transfo_caracsup_if_budget_dict(ligne_budget_dict : dict) -> dict: 
 type_caracsup = ligne_budget_dict.get('CaracSup')  # Permet de connaitre le type de CaracSup
 if type_caracsup is not None:  # Vérifie si la clé 'CaracSup' existe
    if isinstance(type_caracsup, dict):
      dict_caracsup = type_caracsup
      ligne_budget_dict['CaracSup_1_Lib'] = {'@V': dict_caracsup.get('@Code', '')}
      ligne_budget_dict['CaracSup_1_Val'] = {'@V': dict_caracsup.get('@V', '')}
    elif isinstance(type_caracsup, list):
      dict_caracsup = ligne_budget_dict['CaracSup']
      caracsup_propre = {}
      for z, entry in enumerate(dict_caracsup, start=1):
        code = f'CaracSup_{z}_Lib'
        valeur = f'CaracSup_{z}_Val'
        caracsup_propre[code] = entry.get('@Code', '')
        caracsup_propre[valeur] = entry.get('@V', '')
        ligne_budget_dict.update(caracsup_propre)      
 return ligne_budget_dict

In [27]:
chemin_fichiers_xml = '../../etl_hub_coll/fichiers/todo_xml/'

In [51]:
def task_1_avec_les_13_old(chemin) : 
 ''' Traite les XML et les envoies sous format parquet,
   le nettoyage des @V est fait à cette étape'''
 
 liste_fichiers_safe = [f for f in chemin_fichiers_xml if _isolement_id_liste_comp(f) not in _recherche_id_dans_parquet()]
 for fichier in liste_fichiers_safe : 
  try : 
   id_fichier = _isolement_id(fichier)
   dico = parse_fichier(fichier)
   metadonnees = dico['DocumentBudgetaire']['EnTeteDocBudgetaire']
   metadonnees['Id_Fichier'] = {'@V' : id_fichier}
   docbase = dico['DocumentBudgetaire']['Budget']['LigneBudget']
   if isinstance(docbase, list):
    for i in docbase : 
      i.update(metadonnees)
      i = transfo_mtsup_if_dict(i)
      i = transfo_caracsup_if_budget_dict(i)
   elif isinstance(docbase, dict) : 
    docbase.update(metadonnees)
    docbase = transfo_mtsup_if_dict(docbase)
    docbase = transfo_caracsup_if_budget_dict(docbase)
   df_base = pd.DataFrame(docbase)
   df_propre = nettoyage_V(df_base)
   df_propre.to_parquet(f'./fichiers/todo_parquet/{id_fichier}_pyarrow', engine='pyarrow')
   #shutil.move(fichier, './fichiers/done_xml/')
  except Exception as e : 
    print(f'Erreur fichier {fichier}, extraction impossible')
    #shutil.move(fichier, './fichiers/todo_xml/error/')
    print(e)
    continue 

In [32]:
FICHIERS_TODO = glob.glob(os.path.join(chemin_fichiers_xml, "*.gz"))

In [ ]:
liste_comp_test_int = [f for f in FICHIERS_TODO if int(_isolement_id(f)) not in _recherche_id_dans_parquet()]
liste_comp_test_int

In [34]:
liste_comp_test = [f for f in FICHIERS_TODO if _isolement_id(f) not in _recherche_id_dans_parquet()]
liste_comp_test

KeyboardInterrupt: 

_recherche_id_dans_parquet = 30.2sec 

liste_todo = 0sec 

In [42]:
liste_id_parquet = _recherche_id_dans_parquet()
type(liste_id_parquet[1])

int

In [48]:
liste_todo = [f for f in FICHIERS_TODO]
type(liste_todo[1])

str

j'ai forcé l'arrêt de liste_test_complete à 12min, c'est beaucoup trop long. 

Piste de ce qui ne va pas : 

- La boucle for demande trop de recherche, voire si je traite ça sous forme de df pandas / polars. 
- à chaque boucle for, ça ré-appelle la fonction, ce qui fait 20 000 * 30sec, ce qui fait 7j, j'ai pas envie de laisser tourner aussi longtemps juste pour voir si j'ai raison. 

In [44]:
liste_test_complete = [zz for zz in FICHIERS_TODO if int(_isolement_id(zz)) not in _recherche_id_dans_parquet()]

Error: Invalid Error: KeyboardInterrupt: <EMPTY MESSAGE>

At:
  d:\python\Lib\inspect.py(1749): currentframe
  C:\Users\v_enz\AppData\Local\Temp\ipykernel_6624\2259728749.py(4): _recherche_id_dans_parquet
  C:\Users\v_enz\AppData\Local\Temp\ipykernel_6624\2704376875.py(1): <listcomp>
  C:\Users\v_enz\AppData\Local\Temp\ipykernel_6624\2704376875.py(1): <module>
  C:\Users\v_enz\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py(3526): run_code
  C:\Users\v_enz\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py(3466): run_ast_nodes
  C:\Users\v_enz\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py(3284): run_cell_async
  C:\Users\v_enz\AppData\Roaming\Python\Python311\site-packages\IPython\core\async_helpers.py(129): _pseudo_sync_runner
  C:\Users\v_enz\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py(3079): _run_cell
  C:\Users\v_enz\AppData\Roaming\Python\Python311\sit

On va tenter de voir si c'est vrai pour le deuxieme tiret

Resultat : 1sec. c'est donc vrai 

In [49]:
liste_comprehension_t2 = [zz for zz in liste_todo if int(_isolement_id(zz)) not in liste_id_parquet]

In [50]:
liste_comprehension_t2

['../../etl_hub_coll/fichiers/todo_xml\\619027.xml.gz',
 '../../etl_hub_coll/fichiers/todo_xml\\628857.xml.gz',
 '../../etl_hub_coll/fichiers/todo_xml\\682109.xml.gz',
 '../../etl_hub_coll/fichiers/todo_xml\\722795.xml.gz',
 '../../etl_hub_coll/fichiers/todo_xml\\730073.xml.gz',
 '../../etl_hub_coll/fichiers/todo_xml\\748147.xml.gz',
 '../../etl_hub_coll/fichiers/todo_xml\\756300.xml.gz',
 '../../etl_hub_coll/fichiers/todo_xml\\775775.xml.gz',
 '../../etl_hub_coll/fichiers/todo_xml\\807134.xml.gz',
 '../../etl_hub_coll/fichiers/todo_xml\\809162.xml.gz',
 '../../etl_hub_coll/fichiers/todo_xml\\809290.xml.gz',
 '../../etl_hub_coll/fichiers/todo_xml\\819789.xml.gz',
 '../../etl_hub_coll/fichiers/todo_xml\\821461.xml.gz',
 '../../etl_hub_coll/fichiers/todo_xml\\838663.xml.gz']

In [7]:
def task_1_avec_les_13_t2(chemin_xml_entree, chemin_parquet_verif, chemin_sortie) : 
 ''' Traite les XML et les envoies sous format parquet,
   le nettoyage des @V est fait à cette étape'''
 
 liste_verif_parquet = _recherche_id_dans_parquet(chemin_parquet_db= chemin_parquet_verif)
 liste_fichiers_safe = [chemin_fichier for chemin_fichier in chemin_xml_entree if int(_isolement_id(chemin_fichier)) not in liste_verif_parquet]
 for fichier in liste_fichiers_safe : 
  try : 
   id_fichier = _isolement_id(fichier)
   dico = parse_fichier(fichier)
   metadonnees = dico['DocumentBudgetaire']['EnTeteDocBudgetaire']
   metadonnees['Id_Fichier'] = {'@V' : id_fichier}
   docbase = dico['DocumentBudgetaire']['Budget']['LigneBudget']
   if isinstance(docbase, list):
    for i in docbase : 
      i.update(metadonnees)
      i = transfo_mtsup_if_dict(i)
      i = transfo_caracsup_if_budget_dict(i)
   elif isinstance(docbase, dict) : 
    docbase.update(metadonnees)
    docbase = transfo_mtsup_if_dict(docbase)
    docbase = transfo_caracsup_if_budget_dict(docbase)
   df_base = pd.DataFrame(docbase, 
                          )
   df_propre = nettoyage_V(df_base)
   df_propre.to_parquet(f'{chemin_sortie}{id_fichier}_parquet', engine='pyarrow')
   #shutil.move(fichier, './fichiers/done_xml/')
  except Exception as e : 
    print(f'Erreur fichier {fichier}, extraction impossible')
    #shutil.move(fichier, './fichiers/todo_xml/error/')
    print(e)
    continue 

Ca fonctionne, top. maintenant testons avec des pl (ou avec duckdb)

In [67]:
liste_xml_id = [int(_isolement_id(i)) for i in liste_todo]
df_liste_xml_id = pd.DataFrame({'chemin_xml' : liste_todo,'id_xml' : liste_xml_id})
df_liste_xml_id


,chemin_xml,id_xml
0,../../etl_hub_coll/fichiers/todo_xml\612013.xm...,612013
1,../../etl_hub_coll/fichiers/todo_xml\612019.xm...,612019
2,../../etl_hub_coll/fichiers/todo_xml\612023.xm...,612023
3,../../etl_hub_coll/fichiers/todo_xml\612027.xm...,612027
4,../../etl_hub_coll/fichiers/todo_xml\612030.xm...,612030
...,...,...
19995,../../etl_hub_coll/fichiers/todo_xml\840444.xm...,840444
19996,../../etl_hub_coll/fichiers/todo_xml\840448.xm...,840448
19997,../../etl_hub_coll/fichiers/todo_xml\840455.xm...,840455
19998,../../etl_hub_coll/fichiers/todo_xml\840476.xm...,840476


Ca prend 30sec aussi 

In [70]:
conduck = duckdb.connect(database=':memory:', read_only=False)
duck_parquet = conduck.read_parquet('..//../etl_hub_coll/parquet_demonstration.parquet')
requete = conduck.execute(''' 
    SELECT DISTINCT x.chemin_xml
    FROM df_liste_xml_id as x 
    ANTI JOIN duck_parquet as p ON p.Id_Fichier = x.id_xml''')
liste_mega_test = [x[0] for x in requete.fetchall()]
conduck.close()

In [71]:
len(liste_mega_test)

14

Bonus : creation de parquet via schema propre

In [15]:
schema_parquet = pa.schema([
    ('Id_Fichier', pa.int64()),
    ('IdColl', pa.string()),
    ('DteStr', pa.date32()),
    ('LibellePoste', pa.string()),
    ('IdPost', pa.string()),
    ('TypOpBudg', pa.string()), #des 2 et des 1
    ('OpeCpteTiers', pa.string()),
    ('NatCEPL', pa.string()),
    ('Departement', pa.string()), #On oublie pas les 2A,
    ('NatDec', pa.string()), #A terme, sera un pa.dicitonnary
    ('Exer', pa.int16()),
    ('ProjetBudget', pa.string()), #bool mais à verif 
    ('NatVote', pa.string()), #dico aussi
    ('OpeEquip', pa.string()), #necessite recherche
    ('VoteFormelChap', pa.string()), #bool mais à verif 
    ('TypProv', pa.string()), #int ? 
    ('BudgPrec', pa.string()), #int ?
    ('ReprRes', pa.string()), #int ?
    ('NatFonc', pa.string()), #int ?
    ('CodTypBud', pa.string()),
    ('LibelleEtabPal', pa.string()),
    ('IdEtabPal', pa.string()),
    ('LibelleEtab', pa.string()),
    ('IdEtab', pa.string()),
    ('CodColl', pa.string()),
    ('CodInseeColl', pa.string()),
    ('CodBud', pa.string()),
    ('Nomenclature', pa.string()),
    ('Nature', pa.string()),
    ('Fonction', pa.string()),
    ('LibCpte', pa.string()),
    ('ContNat', pa.string()),
    ('ArtSpe', pa.string()), #Doit être corrigé en bool,
    ('ContFon', pa.string()),
    ('CodRD', pa.string()), #Val D ou R,
    ('MtBudgPrec', pa.float32()), #float32 #nullable = True
    ('MtRARPrec', pa.float32()), #float32
    ('MtPropNouv', pa.float32()), #float32
    ('MtPrev', pa.float32()), #float32
    ('CredOuv', pa.int32()), #int32
    ('MtReal', pa.float32()), #float32
    ('MtRAR3112', pa.float32()), #float32
    ('OpBudg', pa.string()),
    ('MtSup', pa.string()),
        ('MtSup_1_Lib', pa.string()),
        ('MtSup_1_Val', pa.string()),
        ('MtSup_2_Lib', pa.string()),
        ('MtSup_2_Val', pa.string()),
        ('MtSup_3_Lib', pa.string()),
        ('MtSup_3_Val', pa.string()),
        ('MtSup_4_Lib', pa.string()),
        ('MtSup_4_Val', pa.string()),
        ('MtSup_5_Lib', pa.string()),
        ('MtSup_5_Val', pa.string()),
        ('MtSup_6_Lib', pa.string()),
        ('MtSup_6_Val', pa.string()),
        ('MtSup_7_Lib', pa.string()),
        ('MtSup_7_Val', pa.string()),
        ('MtSup_8_Lib', pa.string()),
        ('MtSup_8_Val', pa.string()),
        ('MtSup_9_Lib', pa.string()),
        ('MtSup_9_Val', pa.string()),
        ('MtSup_10_Lib', pa.string()),
        ('MtSup_10_Val', pa.string()), #10
        ('CaracSup_1_Lib', pa.string()),
        ('CaracSup_1_Val', pa.string()),
        ('CaracSup_2_Lib', pa.string()),
        ('CaracSup_2_Val', pa.string()),
        ('CaracSup_3_Lib', pa.string()),
        ('CaracSup_3_Val', pa.string()),
        ('CaracSup_4_Lib', pa.string()),
        ('CaracSup_4_Val', pa.string()),
        ('CaracSup_5_Lib', pa.string()),
        ('CaracSup_5_Val', pa.string()),
        ('CaracSup_6_Lib', pa.string()),
        ('CaracSup_6_Val', pa.string()),
        ('CaracSup_7_Lib', pa.string()), #7
        ('CaracSup_7_Val', pa.string()),
        
    ])

In [16]:
df_vide = pa.Table.from_pandas(pd.DataFrame(columns=schema_parquet.names), schema=schema_parquet)
pq.write_table(df_vide, '.././fichiers/parquet_vide_verif')


In [29]:
import traceback


In [63]:
def task_1_v2(chemin_xml_entree, chemin_parquet_verif, chemin_sortie) : 
 ''' La V2 traite en plus :
  - F Le cas où il n'y a qu'une seule ligne budget (pour caracsup et mtsup).
  - F Ajoute des chemins que l'on peut choisir
  - F Ajoute une verification depuis un plus grand parquet, ça pourra se faire via une db plus tôt, quand on en aura une
  - F Sert un café robusta pas fou mais buvable
  - F Ajoute de nouvelles métadonnées, dont l'année d'exercice, la nature de vote, le siret d'etab parfois different du premier siret etc. 
  - En cours, Fait le schema à cette étape pour éviter tout soucis après 
 '''
 chemin_xml_entree_glob = glob.glob(os.path.join(chemin_xml_entree, "*.gz"))
 liste_verif_parquet = _recherche_id_dans_parquet(chemin_parquet_db= chemin_parquet_verif)
 liste_fichiers_safe = [chemin_fichier for chemin_fichier in chemin_xml_entree_glob if int(_isolement_id(chemin_fichier)) not in liste_verif_parquet]
 for fichier in liste_fichiers_safe : 
  try : 
   id_fichier = _isolement_id(fichier)
   dico = parse_fichier(fichier)
   metadonnees = dico['DocumentBudgetaire']['EnTeteDocBudgetaire']
   metadonnees.update(dico['DocumentBudgetaire']['Budget']['BlocBudget'])
   metadonnees.update(dico['DocumentBudgetaire']['Budget']['EnTeteBudget'])
   metadonnees['Id_Fichier'] = {'@V' : id_fichier}
   docbase = dico['DocumentBudgetaire']['Budget']['LigneBudget']
   if isinstance(docbase, list):
    for i in docbase : 
      i.update(metadonnees)
      i = transfo_mtsup_if_dict(i)
      i = transfo_caracsup_if_budget_dict(i)
   elif isinstance(docbase, dict) : 
    docbase.update(metadonnees)
    docbase = transfo_mtsup_if_dict(docbase)
    docbase = transfo_caracsup_if_budget_dict(docbase)
   df_base = pd.DataFrame(docbase)
   df_propre = nettoyage_V(df_base)
   df_propre.to_parquet(f'{chemin_sortie}{id_fichier}_parquet', engine='pyarrow')
   #shutil.move(fichier, './fichiers/done_xml/')
  except Exception as e : 
    print(f'Erreur fichier {fichier}, extraction impossible')
    #shutil.move(fichier, './fichiers/todo_xml/error/')
    print(f'Erreur complète : {repr(e)}')
    traceback.print_exc()
    continue 

In [48]:
chemin_parquet_verif_t = '.././fichiers/parquet_vide_verif'
chemin_sortie_parquet_t =  '.././fichiers/todo_parquet/'

In [75]:
task_1_v2('.././fichiers/todo_xml/test1000/t500/',
          '.././fichiers/parquet_vide_verif',
          '.././fichiers/todo_parquet/')

In [26]:
chemin_test_solo = '.././fichiers/todo_xml/test1000/t500/612019.xml.gz'

On ajoute la task 2 pour voir 

In [78]:
def task_2(dossier_entree, chemin_parquet_global, schema_cust) :
 liste_parquet_t = []
 for parquet in glob.glob(os.path.join(dossier_entree, "*")) : 
  liste_parquet_t.append(parquet)
 with pq.ParquetWriter(f"{chemin_parquet_global}.parquet", schema=schema_cust) as writer:
    for file in liste_parquet_t:
        table = pq.read_table(file, schema=schema_cust)
        writer.write_table(table)

In [81]:
schema_parquet = pa.schema([
    ('Id_Fichier', pa.int64()),
    ('IdColl', pa.string()),
    ('DteStr', pa.date32()),
    ('LibellePoste', pa.string()),
    ('IdPost', pa.string()),
    ('TypOpBudg', pa.string()), #des 2 et des 1
    ('OpeCpteTiers', pa.string()),
    ('NatCEPL', pa.string()),
    ('Departement', pa.string()), #On oublie pas les 2A,
    ('NatDec', pa.string()), #A terme, sera un pa.dicitonnary
    ('Exer', pa.int16()),
    ('ProjetBudget', pa.string()), #bool mais à verif 
    ('NatVote', pa.string()), #dico aussi
    ('OpeEquip', pa.string()), #necessite recherche
    ('VoteFormelChap', pa.string()), #bool mais à verif 
    ('TypProv', pa.string()), #int ? 
    ('BudgPrec', pa.string()), #int ?
    ('ReprRes', pa.string()), #int ?
    ('NatFonc', pa.string()), #int ?
    ('CodTypBud', pa.string()),
    ('LibelleEtabPal', pa.string()),
    ('IdEtabPal', pa.string()),
    ('LibelleEtab', pa.string()),
    ('IdEtab', pa.string()),
    ('CodColl', pa.string()),
    ('CodInseeColl', pa.string()),
    ('CodBud', pa.string()),
    ('Nomenclature', pa.string()),
    ('Nature', pa.string()),
    ('Fonction', pa.string()),
    ('LibCpte', pa.string()),
    ('ContNat', pa.string()),
    ('ArtSpe', pa.string()), #Doit être corrigé en bool,
    ('ContFon', pa.string()),
    ('CodRD', pa.string()), #Val D ou R,
    ('MtBudgPrec', pa.float32()), #float32 #nullable = True
    ('MtRARPrec', pa.float32()), #float32
    ('MtPropNouv', pa.float32()), #float32
    ('MtPrev', pa.float32()), #float32
    ('CredOuv', pa.float32()), #int32
    ('MtReal', pa.float32()), #float32
    ('MtRAR3112', pa.float32()), #float32
    ('OpBudg', pa.string()),
    ('MtSup', pa.string()),
        ('MtSup_1_Lib', pa.string()),
        ('MtSup_1_Val', pa.string()),
        ('MtSup_2_Lib', pa.string()),
        ('MtSup_2_Val', pa.string()),
        ('MtSup_3_Lib', pa.string()),
        ('MtSup_3_Val', pa.string()),
        ('MtSup_4_Lib', pa.string()),
        ('MtSup_4_Val', pa.string()),
        ('MtSup_5_Lib', pa.string()),
        ('MtSup_5_Val', pa.string()),
        ('MtSup_6_Lib', pa.string()),
        ('MtSup_6_Val', pa.string()),
        ('MtSup_7_Lib', pa.string()),
        ('MtSup_7_Val', pa.string()),
        ('MtSup_8_Lib', pa.string()),
        ('MtSup_8_Val', pa.string()),
        ('MtSup_9_Lib', pa.string()),
        ('MtSup_9_Val', pa.string()),
        ('MtSup_10_Lib', pa.string()),
        ('MtSup_10_Val', pa.string()), #10
        ('CaracSup_1_Lib', pa.string()),
        ('CaracSup_1_Val', pa.string()),
        ('CaracSup_2_Lib', pa.string()),
        ('CaracSup_2_Val', pa.string()),
        ('CaracSup_3_Lib', pa.string()),
        ('CaracSup_3_Val', pa.string()),
        ('CaracSup_4_Lib', pa.string()),
        ('CaracSup_4_Val', pa.string()),
        ('CaracSup_5_Lib', pa.string()),
        ('CaracSup_5_Val', pa.string()),
        ('CaracSup_6_Lib', pa.string()),
        ('CaracSup_6_Val', pa.string()),
        ('CaracSup_7_Lib', pa.string()), #7
        ('CaracSup_7_Val', pa.string()),
        
    ])

In [82]:
task_2('.././fichiers/todo_parquet/', '.././fichiers/parquet_500', schema_parquet)

In [87]:
conduck.close()

In [104]:
conduck = duckdb.connect(database=':memory:', read_only=False)
parquet_duck = conduck.read_parquet('..//../etl_hub_coll/fichiers/parquet_500.parquet')

requete_1 = (''' Select * FROM parquet_duck''')
df_result1 = conduck.execute(requete_1).fetchdf()


In [105]:
conduck.close()

In [106]:
df_result1.head(1)

,Id_Fichier,IdColl,DteStr,LibellePoste,IdPost,TypOpBudg,OpeCpteTiers,NatCEPL,Departement,NatDec,...,CaracSup_3_Lib,CaracSup_3_Val,CaracSup_4_Lib,CaracSup_4_Val,CaracSup_5_Lib,CaracSup_5_Val,CaracSup_6_Lib,CaracSup_6_Val,CaracSup_7_Lib,CaracSup_7_Val
0,612019,26780487000017,2019-01-01,CENTRE DES FINANCES PUBLIQUES,078027,None,None,Commune,78,09,...,None,None,None,None,None,None,None,None,None,None


In [110]:
df_1_carac3lib_nan = df_result1[df_result1['CaracSup_3_Lib'] == 'nan']
df_1_carac3lib_nan['Id_Fichier'].unique() 

array([614551, 614751, 616362], dtype=int64)

In [114]:
df_1_carac3lib_nan[df_1_carac3lib_nan['CaracSup_1_Lib'] == 'ProgAutoNum']

,Id_Fichier,IdColl,DteStr,LibellePoste,IdPost,TypOpBudg,OpeCpteTiers,NatCEPL,Departement,NatDec,...,CaracSup_3_Lib,CaracSup_3_Val,CaracSup_4_Lib,CaracSup_4_Val,CaracSup_5_Lib,CaracSup_5_Val,CaracSup_6_Lib,CaracSup_6_Val,CaracSup_7_Lib,CaracSup_7_Val


In [115]:
df_1_carac3lib_nan[df_1_carac3lib_nan['CaracSup_1_Lib'] == 'nan']

,Id_Fichier,IdColl,DteStr,LibellePoste,IdPost,TypOpBudg,OpeCpteTiers,NatCEPL,Departement,NatDec,...,CaracSup_3_Lib,CaracSup_3_Val,CaracSup_4_Lib,CaracSup_4_Val,CaracSup_5_Lib,CaracSup_5_Val,CaracSup_6_Lib,CaracSup_6_Val,CaracSup_7_Lib,CaracSup_7_Val


In [116]:
df_1_carac3lib_nan[df_1_carac3lib_nan['CaracSup_2_Lib'] == 'nan']


,Id_Fichier,IdColl,DteStr,LibellePoste,IdPost,TypOpBudg,OpeCpteTiers,NatCEPL,Departement,NatDec,...,CaracSup_3_Lib,CaracSup_3_Val,CaracSup_4_Lib,CaracSup_4_Val,CaracSup_5_Lib,CaracSup_5_Val,CaracSup_6_Lib,CaracSup_6_Val,CaracSup_7_Lib,CaracSup_7_Val
26891,614551,20003019500123,2020-02-26,CENTRE FINANCES PUBLIQUES NICE MUNICIPALE CFNM,006015,nan,None,EPCI,None,09,...,nan,nan,None,None,None,None,None,None,None,None
26892,614551,20003019500123,2020-02-26,CENTRE FINANCES PUBLIQUES NICE MUNICIPALE CFNM,006015,nan,None,EPCI,None,09,...,nan,nan,None,None,None,None,None,None,None,None
26893,614551,20003019500123,2020-02-26,CENTRE FINANCES PUBLIQUES NICE MUNICIPALE CFNM,006015,nan,None,EPCI,None,09,...,nan,nan,None,None,None,None,None,None,None,None
26894,614551,20003019500123,2020-02-26,CENTRE FINANCES PUBLIQUES NICE MUNICIPALE CFNM,006015,nan,None,EPCI,None,09,...,nan,nan,None,None,None,None,None,None,None,None
26895,614551,20003019500123,2020-02-26,CENTRE FINANCES PUBLIQUES NICE MUNICIPALE CFNM,006015,nan,None,EPCI,None,09,...,nan,nan,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46208,616362,21060083900018,2020-03-20,TRESORERIE PRINCIPALE DE MENTON MONSIEUR LE CO...,006008,nan,nan,Commune,None,09,...,nan,nan,None,None,None,None,None,None,None,None
46209,616362,21060083900018,2020-03-20,TRESORERIE PRINCIPALE DE MENTON MONSIEUR LE CO...,006008,nan,nan,Commune,None,09,...,nan,nan,None,None,None,None,None,None,None,None
46210,616362,21060083900018,2020-03-20,TRESORERIE PRINCIPALE DE MENTON MONSIEUR LE CO...,006008,nan,nan,Commune,None,09,...,nan,nan,None,None,None,None,None,None,None,None
46211,616362,21060083900018,2020-03-20,TRESORERIE PRINCIPALE DE MENTON MONSIEUR LE CO...,006008,nan,nan,Commune,None,09,...,nan,nan,None,None,None,None,None,None,None,None


In [123]:
import numpy as np 

In [125]:
df_1_carac3lib_nan = df_1_carac3lib_nan.replace('nan', 'None')
df_1_carac3lib_nan[df_1_carac3lib_nan['CaracSup_2_Lib'] == 'nan']


,Id_Fichier,IdColl,DteStr,LibellePoste,IdPost,TypOpBudg,OpeCpteTiers,NatCEPL,Departement,NatDec,...,CaracSup_3_Lib,CaracSup_3_Val,CaracSup_4_Lib,CaracSup_4_Val,CaracSup_5_Lib,CaracSup_5_Val,CaracSup_6_Lib,CaracSup_6_Val,CaracSup_7_Lib,CaracSup_7_Val


In [134]:
df_1_carac3lib_nan[df_1_carac3lib_nan['CaracSup_2_Lib'] == 'None']
df_result1 = df_result1.replace('nan', 'None')
df_result1['CaracSup_2_Lib'].value_counts()

CaracSup_2_Lib
None           16661
Section         1923
ProgAutoNum     1130
ProgAutoLib      398
TypOpe            29
Name: count, dtype: int64

In [143]:
df_creouv = df_result1[['Id_Fichier', 'CredOuv']]
df_creouv_Nan = df_creouv[df_result1['CredOuv'].isna()]
df_creouv

,Id_Fichier,CredOuv
0,612019,3000.0
1,612019,350.0
2,612019,5950.0
3,612019,3800.0
4,612019,8500.0
...,...,...
59852,617536,5600.0
59853,617536,28410.0
59854,617536,NaN
59855,617536,NaN


In [149]:
print(df_result1['Departement'].value_counts())

Departement
034    4175
30     2049
64     2040
29     1960
09     1567
       ... 
85       30
39       14
226      13
91        7
08        4
Name: count, Length: 88, dtype: int64
